In [1]:
import numpy as np
import datetime as dt
import pandas as pd
import plotly
#import plotly.plotly as py #esse py faz uso da plataforma da plotly e necessita de uma conta
import plotly.graph_objs as go

In [2]:
from plotly.offline import iplot, init_notebook_mode #o iplot que deve ser usado é esse aqui!
init_notebook_mode(connected=True)

In [ ]:
#gerando fake dataset

In [4]:
dataset = {
    "station_id": [],
    "base_timestamp": [],
    "t_inside": [],
    "t_near": [],
    "t_far": [],
    "n_inside": [],
    "n_near": [],
    "n_far": []
}
n_max = 10000
t_max = 300000
alpha_inside = 0.01
alpha_near = 0.2
alpha_far = 1

gen_rand = lambda x: np.random.randint(0, int(x))

for n_station in range(1,5):
    for n_day in range(1,5):
        for n_hour in range(0, 24):
            timestamp = dt.datetime(2018, 10, n_day, n_hour, 0,0).isoformat()
            
            dataset["station_id"].append(n_station)
            dataset["base_timestamp"].append(timestamp)
            dataset["t_inside"].append(gen_rand(t_max*alpha_inside))
            dataset["t_near"].append(gen_rand(t_max*alpha_near))
            dataset["t_far"].append(gen_rand(t_max*alpha_far))
            dataset["n_inside"].append(gen_rand(n_max*alpha_inside))
            dataset["n_near"].append(gen_rand(n_max*alpha_near))
            dataset["n_far"].append(gen_rand(n_max*alpha_far))

pd_dataset = pd.DataFrame(dataset)

In [ ]:
#variaveis globais

In [5]:
lista_nomes = ['', 'urca', 'botfy', 'copa', 'xereu']
regioes = ['inside', 'near', 'far']
lista_stations = pd_dataset['station_id'].unique()
nr_stations = len(lista_stations)
nr_traces = 3

In [ ]:
#funcoes para gerar componentes dos graficos e menus

In [32]:
def trace_str(regiao, pd_dataset_filtered, station_id_filter):
    return go.Bar(
                x = pd_dataset_filtered['base_timestamp'],
                y = pd_dataset_filtered['t_'+regiao] / pd_dataset_filtered['n_'+regiao],
                name = regiao + ' station ' + str(station_id_filter)
        )


In [33]:
def traces_estacao_i(station_id_filter, pd_dataset):
    pd_dataset_filtered = pd_dataset[pd_dataset["station_id"] == station_id_filter]
    return [trace_str(regiao, pd_dataset_filtered, station_id_filter) for regiao in regioes]


In [34]:
def dict_button(selected_station, nr_stations, nr_traces):
    return dict(label = 'Estacao ' + str(lista_stations[selected_station]) + ' - ' + lista_nomes[lista_stations[selected_station]],
                method = 'update',
                args = [{'visible': [i//nr_traces==selected_station for i in range(nr_stations*nr_traces)]},
                        {'title': 'Estacao ' + str(lista_stations[selected_station]) + ' - ' + lista_nomes[lista_stations[selected_station]]}])


In [35]:
def buttons_dropdown(nr_traces, nr_stations):
    return list([dict_button(selected_station, nr_stations, nr_traces) for selected_station in range(nr_stations)]) 


In [ ]:
#gerando componentes dos graficos e menus e graficos

In [41]:
data = []
for i in lista_stations:
    data = data + traces_estacao_i(i, pd_dataset)

In [43]:
updatemenus = list([
    dict(active=-1,
         buttons=buttons_dropdown(nr_traces, nr_stations),
    )
])

In [45]:
#bar_type = 'group'
bar_type = 'stack'

layout = dict(barmode=bar_type,
              updatemenus=updatemenus)

fig = go.Figure(data=data, 
           layout=layout)

#no luga de chamar py.iplot, chama apenas o iplot que foi importado do plotly.offline
iplot(fig, filename = 'tempo medio X data')